In [2]:
#import cv2
#import cv2
import pandas as pd
import numpy as np
import time
#from pyzbar.pyzbar import decode
from datetime import datetime
import os
import shutil  # ✅ for copying files

# --- Load Initial Data ---
df = pd.read_csv("grocery_df.csv")

# Convert necessary date columns
for col in ['Date_Received', 'Expiration_Date', 'Discount_Date', 'Donation_Date']:
    df[col] = pd.to_datetime(df[col])

# --- Status Function ---
def get_status(row, today):
    if today < row['Discount_Date']:
        return 'Fresh'
    elif row['Discount_Date'] <= today < row['Donation_Date']:
        return 'Discount'
    elif row['Donation_Date'] <= today < row['Expiration_Date']:
        return 'Donation'
    else:
        return 'Expired'

def update_status(df, today):
    df['Status'] = df.apply(lambda row: get_status(row, today), axis=1)
    return df

# --- Table Generator ---
def generate_tables(df):
    # Select relevant columns
    inventory_table = df[['Product_ID', 'Product_Name', 'Catagory', 'Supplier_ID', 'Supplier_Name',
                          'Stock_Quantity', 'Reorder_Level', 'Reorder_Quantity', 'Unit_Price',
                          'Date_Received', 'Discount_Date', 'Donation_Date', 'Expiration_Date',
                          'Shelf_Life_(Days)', 'Shelf_Life_Bin', 'Status']].copy()

    # Convert date columns
    inventory_table.loc[:, 'Discount_Date'] = pd.to_datetime(inventory_table['Discount_Date'], errors='coerce').dt.date
    inventory_table.loc[:, 'Donation_Date'] = pd.to_datetime(inventory_table['Donation_Date'], errors='coerce').dt.date

    # Extract numeric value from Unit_Price (keep symbol for later)
    inventory_table['Numeric_Unit_Price'] = (
        inventory_table['Unit_Price'].astype(str).str.extract(r'(\d+\.?\d*)')[0].astype(float)
    )

    # Format Unit_Price back with ₹ symbol
    inventory_table['Unit_Price'] = '$' + inventory_table['Numeric_Unit_Price'].round(2).astype(str)

    # Create sub-tables
    fresh_table = inventory_table[inventory_table['Status'] == 'Fresh'].copy()
    discount_table = inventory_table[inventory_table['Status'] == 'Discount'].copy()
    donation_table = inventory_table[inventory_table['Status'] == 'Donation'].copy()
    expired_table = inventory_table[inventory_table['Status'] == 'Expired'].copy()

    # Calculate and format discounted price
    discount_table['Discounted_Price'] = '$' + (discount_table['Numeric_Unit_Price'] * 0.7).round(2).astype(str)

    # Columns to drop
    columns_to_remove = ['Reorder_Level', 'Reorder_Quantity', 'Date_Received', 'Shelf_Life_Bin', 'Status']

    # Drop from sub-tables
    discount_table = discount_table.drop(columns=columns_to_remove)
    donation_table = donation_table.drop(columns=columns_to_remove + ['Discount_Date'])
    expired_table = expired_table.drop(columns=columns_to_remove + ['Discount_Date', 'Donation_Date'])

    # Optionally drop numeric helper column from all tables
    inventory_table.drop(columns=['Numeric_Unit_Price'], inplace=True)
    discount_table.drop(columns=['Numeric_Unit_Price'], inplace=True, errors='ignore')
    donation_table.drop(columns=['Numeric_Unit_Price'], inplace=True, errors='ignore')
    expired_table.drop(columns=['Numeric_Unit_Price'], inplace=True, errors='ignore')
    fresh_table.drop(columns=['Numeric_Unit_Price','Status'], inplace=True, errors='ignore')

    date_cols = ['Date_Received', 'Expiration_Date', 'Discount_Date', 'Donation_Date']
    for table in [inventory_table, fresh_table, discount_table, donation_table, expired_table]:
        for col in date_cols:
            if col in table.columns:
                table[col] = pd.to_datetime(table[col], errors='coerce').dt.strftime('%Y-%m-%d')
    
    return inventory_table, fresh_table, discount_table, donation_table, expired_table

# --- Set path of already existing backup folder ---
#backup_dir = r'./inventory/my-app/public'  # Change this to your actual path if needed

# --- Real-Time Simulation with JSON Export + Copy ---
def simulate_realtime_update(interval_sec=10, duration_sec=60):
    global df
    start_time = time.time()
    while (time.time() - start_time) < duration_sec:
        today = pd.Timestamp('2025-05-31')  # fixed date for testing
        df = update_status(df, today)
        inventory_table, fresh_table, discount_table, donation_table, expired_table = generate_tables(df)

        # Save JSON to current directory
        inventory_table.to_json("inventory.json", orient='records')
        fresh_table.to_json("fresh.json", orient='records')
        discount_table.to_json("discount.json", orient='records')
        donation_table.to_json("donation.json", orient='records')
        expired_table.to_json("expired.json", orient='records')
        print("JSON files saved and copied to backup.")
        time.sleep(interval_sec)


        # ✅ Ensure the backup directory exists
        #os.makedirs(backup_dir, exist_ok=True)

'''

        # Copy them to backup folder
        shutil.copy("inventory.json", os.path.join(backup_dir, "inventory.json"))
        shutil.copy("fresh.json", os.path.join(backup_dir, "fresh.json"))
        shutil.copy("discount.json", os.path.join(backup_dir, "discount.json"))
        shutil.copy("donation.json", os.path.join(backup_dir, "donation.json"))
        shutil.copy("expired.json", os.path.join(backup_dir, "expired.json"))
'''
        

# --- Start Real-Time Simulation ---
simulate_realtime_update(interval_sec=5, duration_sec=20)

JSON files saved and copied to backup.
JSON files saved and copied to backup.
JSON files saved and copied to backup.
JSON files saved and copied to backup.


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
